# Imports and Constants

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline
import random
import os
from math import log10

from savingOutputs import *
from loadData import *
from masks import *
from decoding import *
from plots import *
from utility import *
from metrics import *


from_who = "our"   # 2 possibilities : "mohameds" or "our"
maps_folder="brain_maps/"+from_who+"_maps"
masks_folder="masks/"+from_who+"_masks"

SEED = 0
random.seed(SEED)
classes = ['Up', 'Down', 'Right', 'Left']
nb_runs = 12
length_one_modality = nb_runs * len(classes)
subjects_ids = range(1, 24)
n_subjects = len(subjects_ids)
n_individual_perms = 1000
class_labels = [""] * (2 * length_one_modality)
for i in range(length_one_modality * 2):
    class_labels[i] = classes[(i // nb_runs) % len(classes)]
labels = dict()
labels["vis"] = np.array(class_labels[:length_one_modality])
labels["aud"] = np.array(class_labels[length_one_modality:])
labels_same = labels["vis"]

within_modal_tasks_regions = [(["vis"], ["V5_L", "V5_R"]),
                              (["vis"], ["PT_L", "PT_R"]),
                              (["aud"], ["V5_L", "V5_R"]),
                              (["aud"], ["PT_L", "PT_R"])]

cross_modal_tasks_regions = [(["vis", "aud"], ["V5_L", "V5_R"]),
                             (["vis", "aud"], ["PT_L", "PT_R"])]

# Analyses

In [2]:
maps_masked, masks_exist = load_full_data(subjects_ids, length_one_modality, maps_folder, masks_folder, is_from_mohamed=(from_who=="mohameds"))

C_values = 10**np.linspace(-10,10,num=1000)
kernels = ["linear","rbf"]

In [ ]:
acc_group_within_combination = {"linear":np.zeros(len(C_values)),"rbf":np.zeros(len(C_values))}
acc_group_cross_combination = {"linear":np.zeros(len(C_values)),"rbf":np.zeros(len(C_values))}

start_time = time.time()
for i,C in enumerate(C_values) :
    for kernel in kernels :
        scaler = sklearn.preprocessing.StandardScaler()
        classifier = sklearn.svm.SVC(C = C, kernel = kernel, random_state=SEED)  # other parameters are default parameters
        pipeline = Pipeline([('std_scaler', scaler), ('svm', classifier)])
        decoder = Decoder(n_perm=n_individual_perms, model=pipeline, n_classes=len(classes), n_splits=nb_runs, seed=SEED)
        decoder.set_masks_exist(masks_exist)

        confusion_matrixes_within = decoder.within_modality_decoding(maps_masked, labels, subjects_ids, within_modal_tasks_regions)
        group_confusion_matrixes_within = average_dicos(confusion_matrixes_within)

        confusion_matrixes_cross = decoder.cross_modality_decoding(maps_masked, labels, subjects_ids, cross_modal_tasks_regions)
        group_confusion_matrixes_cross = average_dicos(confusion_matrixes_cross)

        # saving results
        params = classifier.get_params()
        out_directory = "out/"+from_who+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
        create_directory(out_directory)

        save_dicts(out_directory + "confusion_matrixes_within.csv", confusion_matrixes_within,
                   list(confusion_matrixes_within[0].keys()), subjects_ids)
        acc_within = compute_metric(out_directory, subjects_ids, {'name': 'accuracy', 'function': accuracy}, "within", masks_exist, ret=True)

        save_dicts(out_directory + "confusion_matrixes_cross.csv", confusion_matrixes_cross,
                   list(confusion_matrixes_cross[0].keys()), subjects_ids)
        acc_cross = compute_metric(out_directory, subjects_ids, {'name': 'accuracy', 'function': accuracy}, "cross", masks_exist, ret=True)

        within_modality_group_results = average_dicos(acc_within)
        save_dicts(out_directory + "group_scores_within.csv", [within_modality_group_results],
                   list(within_modality_group_results.keys()), [0])
        acc_group_within_combination[kernel][i] = within_modality_group_results

        cross_modality_group_results = average_dicos(acc_cross)
        save_dicts(out_directory + "group_scores_cross.csv", [cross_modality_group_results],
                   list(cross_modality_group_results.keys()), [0])
        acc_group_cross_combination[kernel][i] = cross_modality_group_results

        if i%10 == 0 :
            duration = time.time()-start_time
            print(i+"/"+str(len(C_values)*2)+"iterations done in "+str(duration)+" seconds")

# Plot scores depending on parameters

In [ ]:
for analysis in acc_group_within_combination[0]["linear"]:
    # faire le plot
    linear_scores = [score for score in acc_group_within_combination["linear"]]
    rbf_scores = [score for score in acc_group_within_combination["rbf"]]
    plt.plot(C_values,linear_scores,label="linear")
    plt.plot(C_values,rbf_scores,label="gaussian")
    plt_directory = "plots/"+from_who+"_"+"_".join(k+"="+str(params[k]) for k in ['C', 'kernel'])+"/"
    create_directory(plt_directory)
    plotter = Plotter(plt_directory, subjects_ids)
    plotter.save(analysis+" - accuracy depending on C and kernel of SVM","tunnig_parameters","mean accuracy","C",legend=True)